In [76]:
import pandas as pd

In [77]:
predictions_path = "predictions_data_PROCESSED.xlsx"
assignments_path = "In-Program Ptcpts - VRFIDs.xlsx"


predictions_df = pd.read_excel(predictions_path) #, nrows= 1500)
assignments_df = pd.read_excel(assignments_path)

In [90]:


def get_dept_score_by_top_k_preds(predictions_df, assignments_df, k):
    #top_scores_idx = predictions_df.groupby(['SP_ID'])['Skill Score'].idxmax()
    #top_preds_df = predictions_df.loc[top_scores_idx]  
    top_k_preds_df = predictions_df.groupby('SP_ID').apply(lambda x: x.nlargest(k, 'Skill Score')).reset_index(drop=True)
    pred_assigns_join_df = pd.merge(top_k_preds_df, assignments_df, left_on='SP_ID', right_on='SP ID', how='inner')
    pred_assigns_truth_df = pred_assigns_join_df.copy() 
    pred_assigns_truth_k_grouped_df = pred_assigns_truth_df.groupby(['SP_ID', 'Seva Dept']).agg({'Department': list, 'Skill Score': list}).reset_index()
    pred_assigns_truth_k_grouped_df['InTopK'] = pred_assigns_truth_k_grouped_df.apply(lambda row: row['Seva Dept'] in row['Department'], axis=1)
    accuracy = pred_assigns_truth_k_grouped_df['InTopK'].mean()
    return pred_assigns_truth_k_grouped_df, accuracy



In [94]:
dfs = []
scores = []
for k in [10, 20, 30, 40, 50]:
    df, acc = get_dept_score_by_top_k_preds(predictions_df, assignments_df, k)
    dfs.append(df)
    scores.append((k, acc))
    # print(f"Accuracy for top {k} predictions: {acc}")

pd.DataFrame(scores, columns=['k', 'accuracy'])

/var/folders/j6/588zcvgj0sdc9tfsz36sdcp40000gn/T/ipykernel_20708/1093141449.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_k_preds_df = predictions_df.groupby('SP_ID').apply(lambda x: x.nlargest(k, 'Skill Score')).reset_index(drop=True)
/var/folders/j6/588zcvgj0sdc9tfsz36sdcp40000gn/T/ipykernel_20708/1093141449.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_k_preds_df = predictions_df.grou

,k,accuracy
0,10,0.452454
1,20,0.558282
2,30,0.607362
3,40,0.636503
4,50,0.667178
